<a href="https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_income/02_bullet_bonds.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

## Bullet Bonds


A bullet bond is a type of fixed-income security with a very straightforward repayment structure.

### Key Features
1. **No Amortization**  
   - The issuer does not repay any portion of the principal during the life of the bond.  
   - Only interest (coupon) payments are made periodically (e.g., semi-annual or annual).

2. **Principal Repaid at Maturity**  
   - The entire face value (principal) is repaid in one "bullet" payment at the end of the bond’s term.  
   - This makes it different from amortizing bonds (like many mortgages), where principal is gradually repaid.

3. **Predictable Cash Flow**  
   - Investors receive regular coupon payments and then a large lump sum at maturity.  
   - This makes bullet bonds relatively simple to understand and model.

### Advantages
- **For issuers**: Lower cash outflow during the life of the bond since they don’t repay principal until maturity.  
- **For investors**: Predictable stream of income plus a known lump-sum repayment date.

### Risks
- **Credit risk**: Since principal repayment is concentrated at the end, investors bear the risk that the issuer might default before maturity.  
- **Refinancing risk for issuers**: They must have sufficient liquidity or refinancing ability to repay the lump sum when due.  

### Common Examples
- Many **corporate bonds** and **sovereign bonds** are structured as bullet bonds.  
- Eurobonds and U.S. Treasury notes/bonds are classic examples.



In [33]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

from IPython.display import Markdown

bond = FixedRateBullet("2020-01-01", "2025-01-01", 5, 4, notional=100)


def display_bond_cash_flows(cash_flow_dict):
    table = "## Bond Cash Flows Table\n\n"
    table += "| Date       | Cash Flow Value |\n"
    table += "|------------|-----------------|\n"
    for time, value in sorted(cash_flow_dict.items(), key=lambda x: x[0]):
        table += f"| {time.strftime('%Y-%m-%d')} | {value:.2f}            |\n"
    display(Markdown(table))


display_bond_cash_flows(bond.payment_flow)

## Bond Cash Flows Table

| Date       | Cash Flow Value |
|------------|-----------------|
| 2020-04-01 | 1.25            |
| 2020-07-01 | 1.25            |
| 2020-10-01 | 1.25            |
| 2021-01-01 | 1.25            |
| 2021-04-01 | 1.25            |
| 2021-07-01 | 1.25            |
| 2021-10-01 | 1.25            |
| 2022-01-01 | 1.25            |
| 2022-04-01 | 1.25            |
| 2022-07-01 | 1.25            |
| 2022-10-01 | 1.25            |
| 2023-01-01 | 1.25            |
| 2023-04-01 | 1.25            |
| 2023-07-01 | 1.25            |
| 2023-10-01 | 1.25            |
| 2024-01-01 | 1.25            |
| 2024-04-01 | 1.25            |
| 2024-07-01 | 1.25            |
| 2024-10-01 | 1.25            |
| 2025-01-01 | 101.25            |


## Bond Day-Counting Conventions

These rules decide how to count days for calculating bond interest:

- **30/360**: Counts each month as 30 days, year as 360 days. Simple but less accurate. Used in some corporate bonds.
- **30E/360**: Like 30/360, but if a date is the 31st, it’s counted as the 30th. Used in Eurobonds.
- **Actual/Actual**: Counts real days in the period and year (365 or 366). Most accurate. Used in US Treasuries (ISDA) or international bonds (ICMA).
- **Actual/360**: Counts real days in the period, but uses a 360-day year. Common in money markets like loans.
- **Actual/365**: Counts real days in the period, uses a 365-day year. Used in some bond markets.
- **30/365**: Counts each month as 30 days, year as 365 days. Rare, used in specific financial contracts.
- **Actual/Actual-Bond**: Same as Actual/Actual (ICMA), counts real days in period and coupon period (e.g., half-year). Used in bond markets.

In [27]:
from IPython.display import Markdown
import matplotlib.pyplot as plt
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

# Define day-count conventions and their display labels
DAY_COUNT_CONVENTIONS = {
    "30/360": "30/360",
    "30e/360": "30E/360",
    "actual/360": "ACT/360",
    "actual/365": "ACT/365",
    "30/365": "30/365",
    "actual/actual-Bond": "ACT/ACT (Bond)"
}

# Create bonds with different day-count conventions
bonds = {
    f"bond_{i+1}": FixedRateBullet(
        issue_dt="2020-01-01",
        maturity="2025-01-01",
        cpn=5.0,
        cpn_freq=2,
        notional=100,
        day_count_convention=dcc
    )
    for i, (dcc, _) in enumerate(DAY_COUNT_CONVENTIONS.items())
}

def dcc_label(dcc_obj):
    """
    Return a human-readable label for a day-count convention.
    
    Args:
        dcc_obj: String or object representing the day-count convention.
    
    Returns:
        str: Clean label for the day-count convention.
    """
    if isinstance(dcc_obj, str):
        return DAY_COUNT_CONVENTIONS.get(dcc_obj, dcc_obj)
    return DAY_COUNT_CONVENTIONS.get(dcc_obj.__class__.__name__, str(dcc_obj))

def display_bond_accrued_interest(bonds, settlement_date="2023-03-01"):
    """
    Display a Markdown table of accrued interest for bonds.
    
    Args:
        bonds: Dict of bond names to FixedRateBullet objects.
        settlement_date: Date for accrued interest calculation (default: 2023-03-01).
    
    Returns:
        Markdown: Table with bond names, day-count conventions, and accrued interest.
    """
    table = [
        f"## YTM Table (Settlement Date: {settlement_date})\n",
        "| Bond | Day Count Convention | YTM |",
        "|------|---------------------|------------------|"
    ]
    
    for name, bond in bonds.items():
        dcc = getattr(bond, "day_count_convention", None)
        label = dcc_label(dcc) if dcc else "Unknown"
        try:
            ai = bond.yield_to_maturity(price=100)
            ai_str = f"{ai:.6f}"
        except Exception as e:
            ai_str = f"Error: {str(e)}"
        table.append(f"| {name} | {label} | {ai_str} |")
    
    return Markdown("\n".join(table))

# Display the accrued interest table
display(display_bond_accrued_interest(bonds, settlement_date="2023-03-01"))


## YTM Table (Settlement Date: 2023-03-01)

| Bond | Day Count Convention | YTM |
|------|---------------------|------------------|
| bond_1 | DayCount30360() | 0.050000 |
| bond_2 | DayCount30E360() | 0.050000 |
| bond_3 | DayCountActual360() | 0.050000 |
| bond_4 | DayCountActual365() | 0.050000 |
| bond_5 | DayCount30365() | 0.050000 |
| bond_6 | DayCountActualActualBond() | 0.050000 |

### Calculating Bond Price from Discount Curve

The price of a bond is the present value of its future cash flows, which consist of periodic coupon payments and the face value paid at maturity. A discount curve provides the discount factors needed to compute these present values, accounting for the time value of money and interest rate risk. Each cash flow is multiplied by the discount factor corresponding to its payment time, and the sum of these discounted cash flows gives the bond's price.

Consider a 5-year bond with:
- **Face Value**: $100
- **Annual Coupon Rate**: 5% (paid semi-annually, so $2.50 every 6 months)
- **Maturity**: 5 years

##### 1) Identify Cash Flows
- **Coupon Payments**: $2.50 at `t = 0.5, 1.0, ..., 4.5` years (semi-annual).
- **Final Payment**: $102.50 at `t = 5.0` years (face value + final coupon).

##### 2) Apply Discount Curve
Using the `DummyCurve` discount factor, defined as `discount_t(t) = 1 / (1 + 0.05 * t)`, each cash flow is discounted:
- At `t = 0.5`: $2.50 / (1 + 0.05 * 0.5) ≈ $2.439$
- At `t = 5.0`: $102.50 / (1 + 0.05 * 5.0) = $102.50 / 1.25 = $82.0$

##### 3) Calculate Bond Price
Sum the discounted cash flows:
- $2.439 + $2.381 + ... + $82.0 ≈ $102.066$

This total represents the bond's price, reflecting the present value of all future payments adjusted by the discount curve.

In [22]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

bond = FixedRateBullet(issue_dt = '2020-01-01', maturity='2025-01-01', cpn = 5, cpn_freq=1, notional=1000, yield_to_maturity=5, settlement_date="2021-01-01")

print(bond.get_price())



4.488654508629179


## Duration

Duration measures a bond's price sensitivity to changes in interest rates and represents the weighted average time to receive the bond's cash flows. Mathematically, duration approximates the percentage change in a bond's price for a small change in yield, expressed as:

$$D = -\frac{1}{P} \cdot \frac{dP}{dr}$$

The negative sign reflects the inverse relationship between bond prices and yields.

In [16]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

bond_duration_example = FixedRateBullet(issue_dt = '2020-01-01', maturity='2025-01-01', cpn = 5, cpn_freq=1, notional=1000, yield_to_maturity=5, settlement_date="2021-01-01")

### Macaulay Duration

Macaulay duration is the weighted average time until a bond's cash flows are received, expressed in years. It is calculated as:

$$\frac{\sum_{t=1}^n \frac{t \cdot C_t}{(1 + y)^t} + \frac{n \cdot FV}{(1 + y)^n}}{P}$$

where:
- $C_t$: Cash flow (coupon payment) at time $t$
- $FV$: Face value of the bond
- $y$: Yield to maturity (per period)
- $n$: Number of periods until maturity
- $P$: Present value (current price) of the bond

For a zero-coupon bond, since there are no coupon payments ($C_t = 0$), the Macaulay duration simplifies to the time to maturity, $T - t$, where $T$ is the maturity date and $t$ is the current date.

In [18]:
macaulay_duration = bond_duration_example.macaulay_duration()
print(macaulay_duration)

1.1175133907


### Modified Duration

Modified duration directly measures the percentage change in a bond's price for a 1% change in yield. It is derived from the Macaulay duration as:

$$D^{\text{Mod}} = \frac{D^{\text{MC}}}{1 + \frac{y}{k}}$$

where:
- $y$: Annual yield to maturity
- $k$: Number of compounding periods per year (e.g., $k = 2$ for semi-annual, $k = 1$ for annual)

For a zero-coupon bond with annual compounding ($k = 1$), the modified duration is:

$$D^{\text{Mod}} = \frac{T - t}{1 + y}$$


In [20]:
modified_duration = bond_duration_example.modified_duration()

print(modified_duration)

0.3192895402



## Convexity

Convexity measures the sensitivity of a bond’s duration to changes in interest rates and captures the non-linear relationship between bond prices and yields. It is the second derivative of the bond’s price with respect to the yield, scaled by the price:

$$C = \frac{1}{P} \cdot \frac{d^2P}{dr^2}$$

For a zero-coupon bond with annual compounding, the convexity is:

$$C = \frac{(T - t)(T - t + 1)}{(1 + y)^2}$$

In [21]:
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet

convexity_example_bond = FixedRateBullet(issue_dt = '2020-01-01', maturity='2025-01-01', cpn = 5, cpn_freq=1, notional=1000, yield_to_maturity=5, settlement_date="2021-01-01")

convexity = convexity_example_bond.convexity()

print(convexity)

0.162101523


## Effective Duration 

Effective Duration finds the duration for bonds that have options embedded into them. For example a callable bond is an example of a bonds that has an option embedded into it. 

### Spread Effective Duration

The spread effective duration measures how much a bond price changes with respect to a change in the bond's credit spread. 